In [1]:
import torch
import numpy as np
import random

# Set the seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# If you are using CUDA, set the seed for all GPUs
torch.cuda.manual_seed_all(seed)

In [2]:
##################################Inter the Parameters###############################################################

#Hypervector Size
hv_d = 10000

# Number of Epochs
n_epochs = 10

#Epsilon Values
eps = 10

#Number of Rounds
round_needed = 10

#Number of Clients
n_clients = 8

In [3]:
import sklearn
import time
import random
from scipy import stats
import torchhd
import torch
import sys
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import math
import numpy as np
import copy
import random
import joblib
from tqdm import tqdm
import torchvision
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from torch import nn
from torch.optim import Adam
from tqdm import tqdm_notebook
from tabulate import tabulate
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import scipy.misc
from PIL import Image
import skimage
from skimage.color import rgb2gray
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import normalize
import pickle
from sklearn.utils import resample
import torchhd.embeddings as embeddings
from sklearn.metrics import confusion_matrix



In [4]:
import os
import pickle

data_file = 'data_hole_diameter_10.pkl'

# Load data and labels from the specified file
with open(data_file, 'rb') as f:
    all_dataset = pickle.load(f)

In [5]:
batch_data = all_dataset['1']['traindata']
batch_labels = all_dataset['1']['trainlabels']
nClasses = len(np.unique(batch_labels))
nFeatures = batch_data.shape[1]
len_train = len(batch_data)
print(len_train)

11318


In [6]:
batch_size = int(len_train//round_needed)

In [7]:
embed = embeddings.Density(nFeatures, hv_d).cuda()

In [8]:
def Train_HD(data,labels,embed,nClasses,hv_d):
    class_hvs = torch.zeros((nClasses,hv_d), dtype=torch.float64).cuda()
    for i_sub in range(len(data)):
        class_hvs[labels[i_sub]] += torch.sign(embed(data[i_sub]))
        
    return class_hvs

In [9]:
def Update_HD(chv,data,labels,embed,n_epochs):
        
    for i_epoch in range(n_epochs): 
        stop = 1
        len_train_sh = list(range(len(data)))
        random.shuffle(len_train_sh)
        for i_sub in len_train_sh:
            encd_sample =torch.sign(torch.tensor(embed(data[i_sub]), dtype=torch.float64).cuda())
            guess = torch.matmul(chv,encd_sample).argmax()
            if guess!= labels[i_sub]:
                chv[labels[i_sub]] += encd_sample
                chv[guess] -= encd_sample
                stop = 0


        if stop == 1:
            break 
            
    return chv

In [10]:
def Infer(chv,testdata,testlabels,embed):
    
    count = 0
    for i_sub in list(range(len(testdata))):
        encd_sample = torch.sign(torch.tensor(embed(testdata[i_sub]), dtype=torch.float64).cuda())
        guess = torch.matmul(chv,encd_sample).argmax()
        if guess== testlabels[i_sub]:
            count += 1
    test_acc = count/len(testlabels)

    return  test_acc  

In [11]:
def Noise_first_round(chv,eps,hv_d,batch_size):
    
    std_noise = np.sqrt(2*hv_d*np.log(1.25*batch_size))/eps
    
    class_noisy = torch.zeros(chv.shape, dtype=torch.float64).cuda()
    for i_class in range(len(chv)):
        class_noisy[i_class] = chv[i_class] + torch.normal(mean=0, std = std_noise, size=(hv_d,)).cuda()

    return class_noisy 

In [12]:
def Pert_Noise(chv,eps,hv_d,batch_size,rnd,n_clients):
    
    coef_l = (2*hv_d)/(eps**2)
    val_l = np.log((1.25*(rnd-1)*n_clients*batch_size)+(1.25*batch_size))
    real_l = coef_l * val_l
    
    
    coef_r = (2*hv_d)/(n_clients*(eps**2))
    val_r = np.log((1.25*(rnd-2)*n_clients*batch_size)+(1.25*batch_size))
    real_r = coef_r * val_r
    
    sigma_noise = real_l - real_r
    
    class_noisy = torch.zeros(chv.shape, dtype=torch.float64).cuda()
    for i_class in range(len(chv)):
        class_noisy[i_class] = chv[i_class] + torch.normal(mean=0, std = np.sqrt(sigma_noise), size=(hv_d,)).cuda()
        
    return class_noisy 

In [13]:
# Concatenate data from all clients
test_data = []

test_labels = []

for client in all_dataset:
    test_data.append(all_dataset[client]['testdata'])
    test_labels.append(all_dataset[client]['testlabels'])

test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)


# Convert to torch tensors and move to GPU
testdata = torch.tensor(test_data, dtype=torch.float64).cuda()
testlabels = torch.tensor(test_labels, dtype=torch.long).cuda()

In [14]:
print('DP')

print(f"Round {1}")

acc = []

class_hvs_list = []
for client in all_dataset:
    train_data = all_dataset[client]['traindata'][0:batch_size]
    train_labels = all_dataset[client]['trainlabels'][0:batch_size]

    # Convert to torch tensors and move to GPU
    traindata = torch.tensor(train_data, dtype=torch.float64).cuda()
    trainlabels = torch.tensor(train_labels, dtype=torch.long).cuda()
    

    class_hvs = Train_HD(traindata,trainlabels,embed,nClasses,hv_d)    

    class_noisy = Noise_first_round(class_hvs,eps,hv_d,batch_size)
    
    class_hvs_list.append(class_noisy)



class_hvs_stack = torch.stack(class_hvs_list,dim=0)

class_hvs_mean = torch.mean(class_hvs_stack,dim=0)

test_acc = Infer(class_hvs_mean,testdata,testlabels,embed)
print(test_acc)
acc.append(test_acc)

for rnd in range(1,round_needed):
    print(f"Round {rnd+1}")

    
    class_hvs_list = []
    for client in all_dataset:
        train_data = all_dataset[client]['traindata'][rnd*batch_size:(rnd+1)*batch_size]
        train_labels = all_dataset[client]['trainlabels'][rnd*batch_size:(rnd+1)*batch_size]

        # Convert to torch tensors and move to GPU
        traindata = torch.tensor(train_data, dtype=torch.float64).cuda()
        trainlabels = torch.tensor(train_labels, dtype=torch.long).cuda()

        class_hvs = class_hvs_mean.clone()

        class_hvs = Update_HD(class_hvs,traindata,trainlabels,embed,n_epochs)

        class_noisy = Pert_Noise(class_hvs,eps,hv_d,batch_size,rnd+1,n_clients)

        class_hvs_list.append(class_noisy)
    

    class_hvs_stack = torch.stack(class_hvs_list,dim=0)
    class_hvs_mean = torch.mean(class_hvs_stack,dim=0)

    test_acc = Infer(class_hvs_mean,testdata,testlabels,embed)
    print(test_acc)
    acc.append(test_acc)

DP
Round 1
0.41753533568904594
Round 2
0.5229681978798587
Round 3
0.559452296819788
Round 4
0.6000883392226148
Round 5
0.6157685512367491
Round 6
0.6304770318021201
Round 7
0.6790194346289753
Round 8
0.678047703180212
Round 9
0.685821554770318
Round 10
0.7072879858657244


In [49]:

all_predictions = []
all_labels = []


for i_sub in list(range(len(testdata))):
    encd_sample = torch.sign(torch.tensor(embed(testdata[i_sub]), dtype=torch.float64).cuda())
    guess = torch.matmul(class_hvs_mean,encd_sample).argmax()
    all_predictions.append(guess.item())
    all_labels.append(testlabels[i_sub].item())
        
cm = confusion_matrix(all_labels, all_predictions, labels=np.unique(np.asarray(all_labels)))

fnr_per_class = []
fpr_per_class = []

for i in range(nClasses):
    TP = cm[i, i]  # True Positives for class i
    FN = np.sum(cm[i, :]) - TP  # False Negatives for class i
    FP = np.sum(cm[:, i]) - TP  # False Positives for class i
    TN = np.sum(cm) - (TP + FN + FP)  # True Negatives for class i

    # Compute FNR and FPR for class i
    FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0

    fnr_per_class.append(FNR)
    fpr_per_class.append(FPR)

# Macro-average FNR and FPR
macro_fnr = np.mean(fnr_per_class)
macro_fpr = np.mean(fpr_per_class)


print("Per-Class FNR:", fnr_per_class)
print("Per-Class FPR:", fpr_per_class)
print(f"Macro-Averaged FNR: {macro_fnr:.4f}")
print(f"Macro-Averaged FPR: {macro_fpr:.4f}")

Per-Class FNR: [0.2449628844114528, 0.23833510074231176, 0.3947298728813559]
Per-Class FPR: [0.13500264970853207, 0.2178060413354531, 0.08622746553552492]
Macro-Averaged FNR: 0.2927
Macro-Averaged FPR: 0.1463
